In [60]:
#imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [61]:
data = pd.read_csv('MNIST.csv')
data = np.array(data)
np.random.shuffle(data)

In [62]:
m, n = data.shape

data_test = data[0:1000].T
Y_test = data_test[0]
X_test = data_test[1:n]
X_test = X_test / 255

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255

_, m_train = X_train.shape

In [63]:
data_train.shape

(785, 41000)

In [64]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def ReLU_prime(Z):
    return Z > 0

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

# def sig(Z):
#     A = 1 / (1 + sum(np.exp(-Z)))
#     return A

# def sig_prime(Z):
#     A = sig(Z) * (1 - sig(Z))
#     return A


def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)

    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)

    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis = 1).reshape(-1, 1)

    dZ1 = W2.T.dot(dZ2) * ReLU_prime(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis = 1).reshape(-1, 1)
    
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1

    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2

    return W1, b1, W2, b2


In [65]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return round((np.sum(predictions == Y) / Y.size) * 100, 2)

def gradient_decent(X, Y, epochs, alpha):
    W1, b1, W2, b2 = init_params()

    for e in range(epochs):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        if e % 50 == 0:
            print("Epoch: ", e)
            print("Accuracy: ", get_accuracy(get_predictions(A2), Y), "%")

    return W1, b1, W2, b2

In [66]:
W1, b1, W2, b2 = gradient_decent(X_train, Y_train, 10000, 0.05)

Epoch:  0
Accuracy:  8.55 %
Epoch:  50
Accuracy:  28.63 %
Epoch:  100
Accuracy:  42.82 %
Epoch:  150
Accuracy:  50.58 %
Epoch:  200
Accuracy:  55.7 %
Epoch:  250
Accuracy:  59.58 %
Epoch:  300
Accuracy:  63.0 %
Epoch:  350
Accuracy:  66.26 %
Epoch:  400
Accuracy:  69.26 %
